<a href="https://colab.research.google.com/github/valeriamedina2/Regresion-No-lineal-y-Regresion-Logistica/blob/main/2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('Gastos y costos 2023_sin_nulos.csv')
df.head(5)

,Unnamed: 0,Fecha,Folio,UUID,RFC,Proveedor,TIPO GASTO,Descripción,MP,FP,...,Importe,IVA,RET ISR,RET IVA,Otros,TOTAL MX,TOTAL SAT,Tipo,Status,Poliza
0,0,2023-01-01,31,49536321-60A4-523E-A32F-E8BDF5C2276E,AAE6411055P8,AESA ARTICULOS ELECTRICOS,COMPRAS,Pago,--,--,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,P,Vigente,--
1,1,2023-01-01,38,20FBB63B-D550-512F-9668-EEC6F983CEB7,AAE6411055P8,AESA ARTICULOS ELECTRICOS,COMPRAS,Pago,--,--,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,P,Vigente,--
2,2,2023-01-01,341932,DB8C4388-D4E0-4F0C-A09C-77C773257F44,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006876 30/36 de 03/12/...,PPD,99.0,...,5551.38,888.22,0.0,0.0,0.0,6439.60,6439.60,I,Vigente,PE01-ENE23
3,3,2023-01-01,341933,A81A63F5-D5EC-4601-803E-375AF9CC5021,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006877 30/36 de 03/12/...,PPD,99.0,...,5551.98,888.32,0.0,0.0,0.0,6440.30,6440.30,I,Vigente,PE02-ENE23
4,4,2023-01-01,342205,DCC10492-DE51-4C87-8430-5676FBF81931,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000009486 5/48 de 03/12/2...,PPD,99.0,...,8454.37,1352.70,0.0,0.0,0.0,9807.07,9807.07,I,Vigente,PE03-ENE23


In [ ]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['TIPO GASTO'])
unico

array(['--', 'ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'CAPACITACION',
       'COMISION BANCARIA', 'COMISION MIXTA', 'COMPRAS', 'COMPRAS ',
       'CREDITO ', 'DERECHOS', 'FLETES', 'GASOLINA', 'GERENCIA ADMON',
       'HONORARIOS CONTABLES', 'HONORARIOS PM', 'IMSS/INFONAVIT',
       'MAQUINARIA', 'MENSAJERIA', 'MONITOREO CAMARAS ADT', 'MTT OFICINA',
       'MTTO COMPUTO', 'MTTO LOCAL', 'MTTO MAQUINARIA', 'MTTO OFICINA ',
       'MTTO TRANSPORTE', 'NO DEDUCIBLE', 'PAPELERIA', 'PAQUETERIA',
       'PUBLICIDAD', 'RENTA AUTO', 'RENTA COPIADORA', 'SEGUROS',
       'SUELDOS DIRECCION', 'SUSCRIPCIONES', 'TELEFONO', 'UNIFORMES',
       'VALES DESPENSA', 'VARIOS'], dtype=object)

In [ ]:
#Convierto una variable a dicotómica 
df['TIPO GASTO']= df['TIPO GASTO'].replace(['COMPRAS '], "COMPRAS")

In [ ]:
#Convierto una variable a dicotómica 
df['TIPO GASTO']= df['TIPO GASTO'].replace(['--', 'ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'CAPACITACION',
       'COMISION BANCARIA', 'COMISION MIXTA',
       'CREDITO ', 'DERECHOS', 'FLETES', 'GASOLINA', 'GERENCIA ADMON',
       'HONORARIOS CONTABLES', 'HONORARIOS PM', 'IMSS/INFONAVIT',
       'MAQUINARIA', 'MENSAJERIA', 'MONITOREO CAMARAS ADT', 'MTT OFICINA',
       'MTTO COMPUTO', 'MTTO LOCAL', 'MTTO MAQUINARIA', 'MTTO OFICINA ',
       'MTTO TRANSPORTE', 'NO DEDUCIBLE', 'PAPELERIA', 'PAQUETERIA',
       'PUBLICIDAD', 'RENTA AUTO', 'RENTA COPIADORA', 'SEGUROS',
       'SUELDOS DIRECCION', 'SUSCRIPCIONES', 'TELEFONO', 'UNIFORMES',
       'VALES DESPENSA', 'VARIOS'], "OTROS GASTOS")
df

,Unnamed: 0,Fecha,Folio,UUID,RFC,Proveedor,TIPO GASTO,Descripción,MP,FP,...,Importe,IVA,RET ISR,RET IVA,Otros,TOTAL MX,TOTAL SAT,Tipo,Status,Poliza
0,0,2023-01-01,31,49536321-60A4-523E-A32F-E8BDF5C2276E,AAE6411055P8,AESA ARTICULOS ELECTRICOS,COMPRAS,Pago,--,--,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,P,Vigente,--
1,1,2023-01-01,38,20FBB63B-D550-512F-9668-EEC6F983CEB7,AAE6411055P8,AESA ARTICULOS ELECTRICOS,COMPRAS,Pago,--,--,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,P,Vigente,--
2,2,2023-01-01,341932,DB8C4388-D4E0-4F0C-A09C-77C773257F44,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000006876 30/36 de 03/12/...,PPD,99.0,...,5551.38,888.22,0.0,0.0,0.0,6439.60,6439.60,I,Vigente,PE01-ENE23
3,3,2023-01-01,341933,A81A63F5-D5EC-4601-803E-375AF9CC5021,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000006877 30/36 de 03/12/...,PPD,99.0,...,5551.98,888.32,0.0,0.0,0.0,6440.30,6440.30,I,Vigente,PE02-ENE23
4,4,2023-01-01,342205,DCC10492-DE51-4C87-8430-5676FBF81931,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000009486 5/48 de 03/12/2...,PPD,99.0,...,8454.37,1352.70,0.0,0.0,0.0,9807.07,9807.07,I,Vigente,PE03-ENE23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,392,2023-02-01,882974,3E97C399-6747-4BCA-8436-ED4B0933DF33,BIN940223KE0,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",OTROS GASTOS,COMISIONES E INTERESES,PUE,3.0,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,I,Vigente,--
393,393,2023-02-01,053560,B7132758-F800-4797-B1D6-CC61AFFEE444,BPU7901018D4,BEBIDAS PURIFICADAS,OTROS GASTOS,Epura 20 Lts Policarbonato,PUE,1.0,...,220.00,0.00,0.0,0.0,0.0,220.00,220.00,I,Vigente,--
394,394,2023-02-01,1092957089,D1E2DC1E-9FFB-46C6-82E4-B350B83CB6F7,BMN930209927,BANCO MERCANTIL DEL NORTE SA INSTITUCION DE BA...,OTROS GASTOS,SERVICIOS BANCARIOS,PUE,3.0,...,18071.52,682.40,0.0,0.0,0.0,18753.92,18753.92,I,Vigente,--
395,395,2023-02-01,1453820,6249FE27-6FA6-4EFB-910E-3CD67A0E29A1,EFE8908015L3,EFECTIVALE S. de R.L. de C.V.,OTROS GASTOS,CONSUMOS DE COMBUSTIBLE DE MONEDEROS ELECTRONICOS,PUE,5.0,...,14472.32,0.00,0.0,0.0,0.0,14472.32,14472.32,I,Vigente,--


In [ ]:
#Verificar los valores sin repetirse de una columna
unico2 = np.unique(df['TIPO GASTO'])
unico2

array(['COMPRAS', 'OTROS GASTOS'], dtype=object)

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['TOTAL MX']]
Var_Dep= df['TIPO GASTO']

In [ ]:
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[60  0]
 [60  0]]


In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.5


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.6666666666666666
